# Test

## Library Setting

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from models import VGG16

import os
from tqdm import tqdm

/jun/anaconda3/envs/visual_ai/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Model

In [2]:
model = VGG16(num_classes=88, use_pretrain=False)
model_path = os.path.join("checkpoints", "best_model_aug_2405011500_epoch10.pth")
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [3]:
# 데이터셋 경로
dataset_path = os.path.join("mvtec_anomaly_detection_imagefolder", "test")

# 이미지 변환 설정
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet 데이터셋의 평균과 표준편차 사용
                                ])

# 데이터셋 로드
dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=transform)

# 데이터 로더 설정
batch_size = 16
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

# gpu or cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

VGG16(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU()
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), p

## Top1 & Top3 Accuracy

In [4]:
total = 0
class_correct_top1 = [0] * len(dataset.classes)
class_correct_top3 = [0] * len(dataset.classes)

correct_top1 = 0
correct_top3 = 0

class_correct_top3_list = [[0] * len(dataset.classes) for _ in range(len(dataset.classes))]
class_total = [0] * len(dataset.classes)

true_labels = []
predicted_labels = []

with tqdm(total=len(test_loader), desc='Testing', unit='batch') as pbar:
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.topk(outputs, 3, dim=1)  # 상위 3개 클래스 인덱스
            total += labels.size(0)
            for i in range(len(labels)):
                class_total[labels[i]] += 1
                true_labels.append(labels[i].item())
                predicted_labels.append(predicted[i][0].item())  # 모델이 가장 확률이 높은 클래스 선택

                if labels[i] == predicted[i][0]:
                    correct_top1 += 1
                    class_correct_top1[labels[i]] += 1

                if labels[i] in predicted[i]:
                    correct_top3 += 1
                    class_correct_top3[labels[i]] += 1
                    for j in predicted[i]:
                        class_correct_top3_list[labels[i]][j] += 1

            pbar.update(1)
            pbar.set_postfix({'Top-1 Accuracy': correct_top1 / total, 'Top-3 Accuracy': correct_top3 / total})

# F1 점수 계산
from sklearn.metrics import f1_score

f1 = f1_score(true_labels, predicted_labels, average='weighted')
print("Weighted F1 Score:", f1)


Testing: 100%|██████████| 240/240 [02:23<00:00,  1.67batch/s, Top-1 Accuracy=0.567, Top-3 Accuracy=0.842]


Weighted F1 Score: 0.5448512142767702


## Save Results

In [5]:
import csv
import numpy as np

csv_file_path = "class_accuracy.csv"

data = [["Label_Name", "Top1_Correct_Counts", "Top1_Total_Samples", "Top1_Accuracy", "Top3_Indices_1",
         "Top3_Indices_2", "Top3_Indices_3", "Top3_Counts_1", "Top3_Counts_2", "Top3_Counts_3",
         "Top3_Total_Samples", "Top3_Accuracy", "Top3_Probability_1", "Top3_Probability_2", "Top3_Probability_3"]]

for i, label_name in enumerate(dataset.classes):
    # Top-1 정보
    top1_correct_counts = class_correct_top1[i]
    top1_total_samples = dataset.targets.count(i)
    top1_accuracy = top1_correct_counts / top1_total_samples if top1_total_samples > 0 else 0

    # Top-3 정보
    top3_indices = np.argsort(class_correct_top3_list[i])[-3:][::-1]
    top3_correct_counts = np.sort(class_correct_top3_list[i])[-3:][::-1]
    top3_total_samples = np.sum(class_correct_top3_list[i])
    top3_accuracy = class_correct_top3[i] / top1_total_samples if top1_total_samples > 0 else 0

    top3_probability = top3_correct_counts / top3_total_samples

    data.append([label_name, top1_correct_counts, top1_total_samples, top1_accuracy,
                 *top3_indices[:3], *top3_correct_counts[:3], top3_total_samples, top3_accuracy, *top3_probability[:3]])

with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

print(f"top-1 and top-3 accuracy predictions saved to {csv_file_path}")


top-1 and top-3 accuracy predictions saved to class_accuracy.csv
